In [1]:
#ONS - Children living in long-term workless households, by disability status (Table H)
from gssutils import *
import pandas as pd

scraper = Scraper('https://www.ons.gov.uk/employmentandlabourmarket/peoplenotinwork/unemployment/datasets/hchildrenlivinginlongtermworklesshouseholdsandworklesshouseholdsbydisabilitystatus')

In [2]:
#One tab containing 5 tables: a - e. 
tabs = {tab.name: tab for tab in scraper.distribution(latest=True, mediaType=Excel).as_databaker()}

In [3]:
# Each dimension and their options. Identifying them due to the layout of the spreadsheet.
# Might be a better way of doing this ? instead of hard coding options. 
year_options = ['2006.0', '2007.0', '2008.0', '2009.0', '20103', '2011.0', '2012 r', 
                '20134 r', '2014 r', '2015 r', '2016 r', '2017 r', '2018.0']

household_disability_status = ['No disabled adults in household',
                        'Some adults in household are disabled',
                        'All adults in household are disabled', 'Total'] #is total required here ?

workless_household_type = ['Children in long-term workless households1'
                           , 'Children in workless households2'
                           , 'All children5'] 
count_type = ['Thousands', 'percent']

In [4]:
tab = tabs['CILTWH H']
ref_cell = tab.filter('Table a')
ref_cell.assert_one()

{<A3 'Table a'>}

In [5]:
year = ref_cell.fill(DOWN).one_of(year_options)
household_type = tab.excel_ref('B2').fill(DOWN).one_of(workless_household_type)
disability_status = tab.excel_ref('B4').expand(RIGHT).expand(DOWN).one_of(household_disability_status)
count = tab.excel_ref('E2').expand(DOWN).one_of(count_type)
observations = tab.excel_ref('B6').expand(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace().is_number()

In [6]:
Dimensions = [
                HDim(year,'Year',DIRECTLY,LEFT),
                HDim(disability_status,'Household disability status',DIRECTLY,ABOVE),
                HDim(household_type,'Workless Household Type',CLOSEST,ABOVE),
                HDim(count, 'Measure Type', CLOSEST, ABOVE), 
                HDimConst('Unit','People')
            ]

In [7]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
new_table = c1.topandas()
#savepreviewhtml(c1, fname="Preview.html")

In [8]:
#Removed percent values (tables b, d) as this can be derived. 
new_table = new_table[~new_table['Measure Type'].isin(['percent'])]
new_table.rename(columns={'OBS': 'Count'}, inplace=True)
new_table['Count'] = new_table['Count'].astype(int)
new_table['Year'] = new_table['Year'].str[:4]
new_table['Year'] = new_table['Year'].apply(lambda x: pd.to_numeric(x, downcast='integer'))
new_table['Workless Household Type'] = new_table['Workless Household Type'].str[:-1]

In [9]:
new_table

,Count,Year,Household disability status,Workless Household Type,Measure Type,Unit
0,959098,2006,No disabled adults in household,Children in long-term workless households,Thousands,People
1,280765,2006,Some adults in household are disabled,Children in long-term workless households,Thousands,People
2,304317,2006,All adults in household are disabled,Children in long-term workless households,Thousands,People
3,1544180,2006,Total,Children in long-term workless households,Thousands,People
4,972888,2007,No disabled adults in household,Children in long-term workless households,Thousands,People
5,291156,2007,Some adults in household are disabled,Children in long-term workless households,Thousands,People
6,290042,2007,All adults in household are disabled,Children in long-term workless households,Thousands,People
7,1554086,2007,Total,Children in long-term workless households,Thousands,People
8,953774,2008,No disabled adults in household,Children in long-term workless households,Thousands,People
9,312141,2008,Some adults in household are disabled,Children in long-term workless households,Thousands,People


In [10]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)
new_table.to_csv(destinationFolder / ('main_to_.csv'), index = False)